Collects results MSE and R2 and create report

In [2]:
import pandas as pd
import os
def report(WINDOW, PARAM):
    reps = os.listdir(f"reports_w{WINDOW}")
    k = []
    df = pd.DataFrame()
    for i in reps:
        mname = i.split(".")[0]
        dfix = pd.read_csv(f"reports_w{WINDOW}\\{i}")
        dfix = dfix[dfix["Unnamed: 0"]==PARAM]
        dfix["model"] = [mname]
        dfix["metric"] = dfix["Unnamed: 0"]
        dfix["window"] = WINDOW
        dfix = dfix.drop("Unnamed: 0", axis=1)
        dfix = dfix.set_index(["model", "metric", "window"])
        dfix = dfix.sort_index()
        df = pd.concat([df, dfix])
        k = dfix
    print("Window ", WINDOW, ".", "Statistic: ", PARAM)
    return df

In [17]:
def summary_on_window(WINDOW):
    df = report(WINDOW, "R2")
    dfms = report(WINDOW, "MSE")
    print("\n")
    df = df[df > 0.1]
    g = df.idxmax().dropna()
    r = df.max().dropna()
    sdf = []
    for i in range(len(g)):
        sdf.append([g.index[i], g[i][0], r[i], dfms.loc[(g[i][0], "MSE", 1)][g.index[i]]])
    r = pd.DataFrame(sdf, columns=["Параметр", "Модель", "R2", "MSE"])
    return r

Window  1 . Statistic:  R2
Window  1 . Statistic:  MSE




,Параметр,Модель,R2,MSE
0,Лимфоциты#_dinam_fact,Faltten1Layer,0.824537,0.001349
1,АСТ_dinam_fact,Faltten1Layer,0.449263,0.000157
2,ЧДД_dinam_fact,Flatten5Layers_batch4,0.344241,0.000147
3,Билирубин общий_dinam_fact,Flatten3Layers,0.397709,0.001319
4,MPV- Средн.объем тромбоцитов_dinam_fact,Flatten5Layers_batch4,0.652261,0.005411
5,PCT- Тромбокрит_dinam_fact,Faltten1Layer,0.314895,0.008465
6,Лимфоциты%_dinam_fact,Faltten1Layer,0.581243,0.009657
7,снижение_сознания_dinam_fact,Flatten5Layers_Mixed_batch4,0.910550,0.005053
8,Лактатдегидрогеназа_dinam_fact,Faltten1Layer,0.191407,0.000576
9,PDW- Индекс расп.по объему тр._dinam_fact,Flatten4Layers,0.642589,0.004587


Window 2

In [26]:
report(2, "MSE")

,,Температура_dinam_fact,Лимфоциты#_dinam_fact,АСТ_dinam_fact,ЧСС_dinam_fact,ЧДД_dinam_fact,Билирубин общий_dinam_fact,MPV- Средн.объем тромбоцитов_dinam_fact,PCT- Тромбокрит_dinam_fact,Лимфоциты%_dinam_fact,снижение_сознания_dinam_fact,Cтепень тяжести по КТ_dinam_fact,Лактатдегидрогеназа_dinam_fact,PDW- Индекс расп.по объему тр._dinam_fact
model,metric,,,,,,,,,,,,,
Faltten1Layer,MSE,0.001972,0.000240,0.000408,0.011614,0.000485,0.001956,0.005194,0.004477,0.008642,0.013124,0.057050,0.000390,0.005206
Flatten3Layers,MSE,0.001822,0.000424,0.000841,0.011566,0.000295,0.001604,0.006457,0.006746,0.010754,0.013347,0.055782,0.000562,0.007241
Flatten3Layers_Mixed,MSE,0.001900,0.000618,0.000331,0.007670,0.000314,0.001296,0.008461,0.008024,0.010744,0.014787,0.060398,0.000594,0.006072
GRU13,MSE,0.001871,0.000224,0.000400,0.011437,0.000277,0.001432,0.005355,0.005527,0.008769,0.013186,0.055522,0.000576,0.005656
GRU13_Batch2,MSE,0.001650,0.000093,0.001032,0.010885,0.000334,0.001950,0.004685,0.004564,0.008470,0.013300,0.055516,0.001528,0.005088
GRU13_Batch4,MSE,0.001694,0.000105,0.000241,0.010718,0.000162,0.001927,0.004663,0.004353,0.008631,0.013321,0.055402,0.000401,0.005496
GRU13_Batch8,MSE,0.001617,0.000092,0.000225,0.010963,0.000218,0.001361,0.004736,0.004477,0.008601,0.013296,0.056015,0.000358,0.005153
GRU32xGRU32_Batch4,MSE,0.001550,0.000434,0.000171,0.010977,0.000156,0.001678,0.004539,0.004105,0.008831,0.013174,0.055960,0.000375,0.005095
GRU60Mixed,MSE,0.001716,0.000345,0.000527,0.007531,0.000277,0.000989,0.007273,0.007033,0.009594,0.013758,0.057833,0.000481,0.004663


Window2

Window 3

Window 4